In [ ]:
#Setting 3 in the paper
import numpy as np
import torch
import model.con_Lasso_layer as con_Lasso_layer
import utils.utils as utils
import scipy.stats as stats
import scipy.sparse as sparse

device = torch.device("cuda:0")

In [ ]:
n = 10
total_number_batch = 200
sample_size = n+1

mean = np.zeros(n)

#generate a sparse precision matrix, s.t. the summation of each row is 0; the absolute of off-diagonal elements <= 0.2
rvs = stats.uniform().rvs    
sparse_matrix = sparse.random(n, n, density=0.4, data_rvs=rvs)   
upper_sparse_matrix = sparse.triu(sparse_matrix)
precision = upper_sparse_matrix + upper_sparse_matrix.T - sparse.diags(sparse_matrix.diagonal())
precision = precision.todense()
precision = -precision*0.1

diag = np.arange(n)
row_sum = np.sum(precision, axis = 0)
precision[diag, diag] = -row_sum
#use precision matrix to generate covariance matrix
cov = np.linalg.inv(precision)

batch = []
for i in range(total_number_batch):
  sample = np.random.multivariate_normal(mean, cov, sample_size)
  sample_cov = np.cov(sample, rowvar = False)
  batch.append(sample_cov)
  
batch = np.array(batch)
batch_array = batch.copy()
batch = torch.from_numpy(batch)

output = batch[1:,::]

#Cholesky Decompositon
L = []
for i in range(total_number_batch-1):
  L.append(np.linalg.cholesky(batch_array[i,::]))
L = np.array(L)
L = torch.from_numpy(L)

batch = batch.unsqueeze(1)
batch = batch.repeat(1,n,1,1)

L = L.unsqueeze(1)
L = L.repeat(1,n,1,1)

Sigma = batch[:(total_number_batch-1),::]

b = (-torch.eye(n).to(device)).unsqueeze(0)
b = b.repeat(total_number_batch-1, 1, 1)

data = {}
data['Sigma'] = Sigma.float()
data['b'] = b.float()
data['L'] = L.float()
data['output'] = output.float()

In [ ]:
#create model
config_model = {
    "hparams": {
        "dim": n,
        "max_abs": 0.2,
        "lr": 1e-3,
        "lam": 0.1,
        "para_method_Sigma": 1,
        "para_method_b": 1 
    },
}
model = con_Lasso_layer.Layer(config_model)

trainer_cvx = utils.Trainer(model, data, device, batch_size = 1, max_epochs = 1, ite_val = 10)
trainer_cvx.prepare_data()

In [ ]:
#test without parameterized method
trainer_cvx.test(load_state = False)

In [ ]:
#train model
trainer_cvx.fit()

In [ ]:
#test model
trainer_cvx.test()